<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Mistral_Llama_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pypdf

In [4]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [5]:
!pip install langchain_community

In [6]:
!pip install sentence_transformers

In [7]:
!pip install llama-index==0.9.39

In [8]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [9]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [10]:
documents = SimpleDirectoryReader("/content/Data").load_data()

In [11]:
documents

[Document(id_='9cdc769d-f30a-49d1-8bb6-699baae003b9', embedding=None, metadata={'page_label': '1', 'file_name': 'AttentionISAllYouNeed.pdf', 'file_path': '/content/Data/AttentionISAllYouNeed.pdf', 'file_type': 'application/pdf', 'file_size': 2128686, 'creation_date': '2024-11-09', 'last_modified_date': '2024-11-09', 'last_accessed_date': '2024-11-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle 

In [12]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [14]:
import torch
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature":0.0,"do_sample":False},
    system_prompt=system_prompt,
    tokenizer_name='mistralai/Mistral-7B-Instruct-v0.1',
    model_name='mistralai/Mistral-7B-Instruct-v0.1',
    device_map="auto",
    model_kwargs={"torch_dtype":torch.float16,"load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [16]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [17]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [18]:
query_engine = index.as_query_engine()
response = query_engine.query("What is Self Attention Mechanism?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Self-attention is a mechanism used in natural language processing and machine learning to identify and weight the importance of different words or phrases in a given context. It is used to model long-distance dependencies between words and to capture the relationships between them. In the context of the provided text, self-attention is used in the encoder self-attention in layer 5 of 6 to identify and weight the importance of different words in the sentence "It is in this spirit that a majority of American governments have passed new laws since 2009 making the registration or voting process more difficult." The attention mechanism allows the model to focus on specific words or phrases that are important for understanding the meaning of the sentence.
